In [ ]:
'''
time-warping è una tecnica utilizzata per allineare i dati temporali di diverse 
dimensioni o durate per rendere più facile confrontarli e analizzarli. 
In questo contesto, il time-warping serve a uniformare i dati di scrittura di 
lettere per fare in modo che le differenze temporali tra le varie ripetizioni 
della stessa lettera siano ridotte, facilitando così la modellizzazione e l’analisi.

L’obiettivo finale è utilizzare i dati allineati temporaneamente per inizializzare 
un HMM, che è un modello statistico usato per descrivere sequenze temporali di dati.
 Un HMM può essere utilizzato per etichettare i dati, cioè per associare 
 le osservazioni (i dati neurali) a stati nascosti (come i diversi caratteri scritti)

 Un Modello di Markov Nascosto (HMM, Hidden Markov Model) è un tipo di modello 
 statistico utilizzato per descrivere sistemi che evolvono nel tempo e generano 
 sequenze di osservazioni. È chiamato “nascosto” perché il modello presuppone 
 che ci siano stati nascosti che non possiamo osservare direttamente, 
 ma che influenzano le osservazioni che possiamo misurare.
'''


In [ ]:

import os
import numpy as np
import scipy.io
import scipy.ndimage
import matplotlib.pyplot as plt
from characterDefinitions import getHandwritingCharacterDefinitions

# Punto verso la directory del dataset
rootDir = os.path.expanduser('~') + '/handwritingBCIData/'

# Imposta GPU
os.environ["CUDA_VISIBLE_DEVICES"] = '0'

# Sessioni da processare
dataDirs = ['t5.2019.05.08', 't5.2019.11.25', 't5.2019.12.09', 't5.2019.12.11', 't5.2019.12.18',
            't5.2019.12.20', 't5.2020.01.06', 't5.2020.01.08', 't5.2020.01.13', 't5.2020.01.15']

# Definizione dei caratteri
charDef = getHandwritingCharacterDefinitions()

# Crea la directory per salvare i dati allineati
if not os.path.isdir(rootDir + 'LSTMTrainingSteps/Step1_TimeWarping'):
    os.mkdir(rootDir + 'LSTMTrainingSteps/Step1_TimeWarping')

# Normalizzazione dei dati senza time-warping
for dataDir in dataDirs:
    
    print('Processing dataset: ' + dataDir)
    dat = scipy.io.loadmat(rootDir + 'Datasets/' + dataDir + '/singleLetters.mat')

    # Normalizzazione dei dati
    for char in charDef['charList']:
        neuralCube = dat['neuralActivityCube_' + char].astype(np.float64)

        # Ottenere gli indici dei blocchi
        trlIdx = [t for t in range(dat['characterCues'].shape[0]) if dat['characterCues'][t, 0] == char]
        blockIdx = dat['blockNumsTimeSeries'][dat['goPeriodOnsetTimeBin'][trlIdx]]
        blockIdx = np.squeeze(blockIdx)

        # Normalizzazione per blocchi
        for b in range(dat['blockList'].shape[0]):
            trialsFromThisBlock = np.squeeze(blockIdx == dat['blockList'][b])
            neuralCube[trialsFromThisBlock, :, :] -= dat['meansPerBlock'][np.newaxis, b, :]
        neuralCube = neuralCube / dat['stdAcrossAllData'][np.newaxis, :, :]

        dat['neuralActivityCube_' + char] = neuralCube

    alignedDat = {}

    # Allineamento temporale con smooth e visualizzazione senza time-warping
    for char in charDef['charList']:
        print(f'Processing character: {char}')

        # Smooting con filtro gaussiano
        smoothed_spikes = scipy.ndimage.gaussian_filter1d(dat['neuralActivityCube_' + char], 3.0, axis=1)

        # Visualizzazione (opzionale)
        plt.figure(figsize=(10, 4))
        plt.plot(smoothed_spikes[:, :, 0])  # Traccia la prima dimensione
        plt.title(f'Smoothed data for {char}')
        plt.show()

        alignedDat[char] = smoothed_spikes
    
    # Salvataggio dei dati normalizzati
    fileName = rootDir + 'LSTMTrainingSteps/Step1_TimeWarping/' + dataDir + '_warpedCubes.mat'
    print('Saving ' + fileName)
    scipy.io.savemat(fileName, alignedDat)